In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# For statistical test
from scipy.stats import chisquare, chi2_contingency ## Stastical test
from scipy.stats import probplot
from scipy import stats
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


# Machine learning libary
from sklearn import metrics
from mlxtend.plotting import plot_confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, precision_recall_fscore_support,roc_curve
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
data = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")

In [ ]:
duplicates = data[data.duplicated()]
print("Duplicate Rows : ",len(duplicates))
duplicates.head()

Duplicate Rows :  24206


,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
1242,1.0,1.0,1.0,1.0,27.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,0.0,30.0,1.0,0.0,10.0,4.0,5.0
1563,0.0,0.0,0.0,1.0,21.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,6.0,8.0
2700,0.0,0.0,0.0,1.0,32.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,6.0,8.0
3160,0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,6.0,8.0
3332,0.0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,9.0,6.0,8.0


In [ ]:
data.drop_duplicates(inplace = True)

In [ ]:
#Pearson Feature Selection
corr_data = data.copy()
cor = corr_data.corr()
cor_target = abs(cor["Diabetes_binary"])
#Selecting highly correlated features
pearson_features = cor_target[cor_target>=0.1]
pearson_features

Diabetes_binary         1.000000
HighBP                  0.254318
HighChol                0.194944
BMI                     0.205086
HeartDiseaseorAttack    0.168213
PhysActivity            0.100404
GenHlth                 0.276940
PhysHlth                0.156211
DiffWalk                0.205302
Age                     0.177263
Education               0.102686
Income                  0.140659
Name: Diabetes_binary, dtype: float64

In [ ]:
#X = data[pearson_features.index.drop('Diabetes_binary')]
X = data.drop('Diabetes_binary', axis = 1)
Y = data['Diabetes_binary']
len(X)

229474

In [ ]:
#from imblearn.under_sampling import NearMiss
#nm = NearMiss(version = 1 , n_neighbors = 10)
#x,y = nm.fit_resample(X,Y)
from imblearn.over_sampling import SMOTE

sm = SMOTE(k_neighbors=10)
x, y = sm.fit_resample(X, Y)

X_train , X_test , Y_train , Y_test = train_test_split(x,y, test_size=0.2 , random_state=42)
len(X_train)

311003

In [ ]:
#from sklearn.preprocessing import RobustScaler
#rb_sc = RobustScaler()
#scaler = rb_sc.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

The analysis below is continue from previous notebook. Each of approved model will be evaluated through execution time.

In [ ]:
#LightGBM
from time import perf_counter, sleep

start = perf_counter()
best_model1 = LGBMClassifier()
best_model1.fit(X_train,Y_train)
Y_pred1 = best_model1.predict(X_test)
accuracy1 = metrics.accuracy_score(Y_test, Y_pred1)
precision_positive1 = metrics.precision_score(Y_test, Y_pred1, pos_label=1)
precision_negative1 = metrics.precision_score(Y_test, Y_pred1, pos_label=0)
recall_sensitivity1 = metrics.recall_score(Y_test, Y_pred1, pos_label=1)
recall_specificity1 = metrics.recall_score(Y_test, Y_pred1, pos_label=0)
f1_positive1 = metrics.f1_score(Y_test, Y_pred1, pos_label=1)
f1_negative1 = metrics.f1_score(Y_test, Y_pred1, pos_label=0)
end = perf_counter()
print(f"Time taken to execute code : {end-start}")

Time taken to execute code : 9.077730171999974


In [ ]:
#Random Forest
start = perf_counter()
best_model2 = RandomForestClassifier()
best_model2.fit(X_train,Y_train)
Y_pred2 = best_model2.predict(X_test)
accuracy2 = metrics.accuracy_score(Y_test, Y_pred2)
precision_positive2 = metrics.precision_score(Y_test, Y_pred2, pos_label=1)
precision_negative2 = metrics.precision_score(Y_test, Y_pred2, pos_label=0)
recall_sensitivity2 = metrics.recall_score(Y_test, Y_pred2, pos_label=1)
recall_specificity2 = metrics.recall_score(Y_test, Y_pred2, pos_label=0)
f1_positive2 = metrics.f1_score(Y_test, Y_pred2, pos_label=1)
f1_negative2 = metrics.f1_score(Y_test, Y_pred2, pos_label=0)
end = perf_counter()
print(f"Time taken to execute code : {end-start}")

Time taken to execute code : 53.720728231000066


In [ ]:
#LightGBM Model with random forest feature selection
rf2 = RandomForestClassifier(random_state = 1, max_features = 'sqrt', n_jobs = 1, verbose = 1)
rf2.fit(X_train, Y_train)
feature = pd.Series(rf2.feature_importances_, index = X_train.columns).sort_values(ascending = False)
rf_features=list(feature[feature>0.025].index)

Y1 = data['Diabetes_binary']
X1 = data[rf_features]

sm = SMOTE(k_neighbors=10)
x1, y1 = sm.fit_resample(X1, Y1)

X1_train , X1_test , Y1_train , Y1_test = train_test_split(x1,y1, test_size=0.2 , random_state=42)

start = perf_counter()
best_model3 = LGBMClassifier()
best_model3.fit(X1_train,Y1_train)
Y_pred3 = best_model3.predict(X1_test)
accuracy3 = metrics.accuracy_score(Y1_test, Y_pred3)
precision_positive3 = metrics.precision_score(Y1_test, Y_pred3, pos_label=1)
precision_negative3 = metrics.precision_score(Y1_test, Y_pred3, pos_label=0)
recall_sensitivity3 = metrics.recall_score(Y1_test, Y_pred3, pos_label=1)
recall_specificity3 = metrics.recall_score(Y1_test, Y_pred3, pos_label=0)
f1_positive3 = metrics.f1_score(Y1_test, Y_pred3, pos_label=1)
f1_negative3 = metrics.f1_score(Y1_test, Y_pred3, pos_label=0)
end = perf_counter()
print(f"Time taken to execute code : {end-start}")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   56.6s finished


Time taken to execute code : 7.391513810000106


In [ ]:
# LightGBM
evaluation_model1 = pd.DataFrame({'Model': ['LightGBM'],
                           'Accuracy': accuracy1,
                           'Precision Positive': precision_positive1,
                           'Precision Negative': precision_negative1,
                           'Recall Sensitivity': recall_sensitivity1,
                           'Recall Specitivity': recall_specificity1,
                           'F1 Positive': f1_positive1,
                           'F1 Negative': f1_negative1
                           })


evaluation_model1

,Model,Accuracy,Precision Positive,Precision Negative,Recall Sensitivity,Recall Specitivity,F1 Positive,F1 Negative
0,LightGBM,0.909519,0.967716,0.864339,0.847042,0.971821,0.903367,0.914935


In [ ]:
# Random Forest
evaluation_model2 = pd.DataFrame({'Model': ['RandomForest'],
                           'Accuracy': accuracy2,
                           'Precision Positive': precision_positive2,
                           'Precision Negative': precision_negative2,
                           'Recall Sensitivity': recall_sensitivity2,
                           'Recall Specitivity': recall_specificity2,
                           'F1 Positive': f1_positive2,
                           'F1 Negative': f1_negative2
                           })

evaluation_model2

,Model,Accuracy,Precision Positive,Precision Negative,Recall Sensitivity,Recall Specitivity,F1 Positive,F1 Negative
0,RandomForest,0.907461,0.95,0.87238,0.859921,0.954868,0.902719,0.911762


In [ ]:
#LightGBM with random forest feature selection.
evaluation_model3 = pd.DataFrame({'Model': ['LightGBM'],
                           'Accuracy': accuracy3,
                           'Precision Positive': precision_positive3,
                           'Precision Negative': precision_negative3,
                           'Recall Sensitivity': recall_sensitivity3,
                           'Recall Specitivity': recall_specificity3,
                           'F1 Positive': f1_positive3,
                           'F1 Negative': f1_negative3
                           })

evaluation_model3

,Model,Accuracy,Precision Positive,Precision Negative,Recall Sensitivity,Recall Specitivity,F1 Positive,F1 Negative
0,LightGBM,0.90804,0.966697,0.86262,0.844929,0.970974,0.901721,0.913595


Based on the execution time and evaluations metric, since the evaluation metrics is not significant for three model, we choose to find the best model with lowest execution time. The lowest execution time is LightGBM Model with random forest feature selection.

# Hyperparameter tuning of the best model (LightGBM Model with random forest feature selection)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
lgb=LGBMClassifier()
parameters1 = {'num_leaves':[20,60,100], 'min_child_samples':[5,15,20],'max_depth':[-1,5,15],
             'learning_rate':[0.05,0.1],'reg_alpha':[0,0.01,0.03]}

lgb_rand=RandomizedSearchCV(lgb,parameters1,scoring='accuracy',n_iter=10,  random_state = 42, cv=10, n_jobs = -1)
lgb_rand.fit(X=X1_train, y=Y1_train)
print(lgb_rand.best_params_)
Y_pred=lgb_rand.predict(X1_test)
print('Classification of the result is:')
print(metrics.accuracy_score(Y1_test, Y_pred))

{'reg_alpha': 0.01, 'num_leaves': 60, 'min_child_samples': 15, 'max_depth': -1, 'learning_rate': 0.1}
Classification of the result is:
0.9093387866394002


Use the parameter for the best model for deployment. End of this notebook, continue to deployment notebook.

In [ ]:
X.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [ ]:
rf_features

['HighBP',
 'GenHlth',
 'HighChol',
 'BMI',
 'Age',
 'Income',
 'Education',
 'Sex',
 'DiffWalk',
 'PhysActivity',
 'Smoker',
 'PhysHlth',
 'HeartDiseaseorAttack',
 'Fruits',
 'MentHlth']

Testing using dummy value random integer

In [ ]:
import random
test = []
for i in range(1000):

  HighBP= random.randint(0,1)
  GenHlth=random.randint(1,5)
  HighChol=random.randint(0,1)
  BMI=random.randint(18,60)
  Age=random.randint(1,13)
  Income=random.randint(1,8)
  Education=random.randint(1,6)
  Sex=random.randint(0,1)
  DiffWalk=random.randint(0,1)
  PhysActivity=random.randint(0,1)
  Smoker=random.randint(0,1)
  PhysHlth=random.randint(0,30)
  HeartDiseaseorAttack=random.randint(0,1)
  Fruits=random.randint(0,1)
  MentHlth=random.randint(0,30)

  dummy = [HighBP,GenHlth,HighChol,BMI,Age,Income,Education,Sex,DiffWalk,PhysActivity,Smoker,PhysHlth,HeartDiseaseorAttack,Fruits,MentHlth]

  dummy = np.reshape(dummy,(1,-1))
  Y_pred = best_model3.predict(dummy)
  test.append(Y_pred[0])

count_diabetic = 0
count_non_diabetic = 0
for i in test:
  if i == 1:
    count_diabetic += 1
  else:
    count_non_diabetic += 1

print("diabetic:", count_diabetic)
print("non diabetic", count_non_diabetic)

diabetic: 108
non diabetic 892
